In [21]:
%load_ext autoreload
%autoreload 2
import torch
from torch.nn import Linear, Sequential, ReLU
from src.bayesian.core import *
from src.bayesian.core import _IMPLEMENTED_BAYESIAN_MODULES
from src.inference.vi import VariationalInference
from src.models.base import Model, ClassifierMixin
from pytorch_lightning import Trainer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
class Net(Model, Sequential, ClassifierMixin):
    pass
        
net = Net(Linear(4, 10), ReLU(), Linear(10, 3))

In [23]:
from sklearn.datasets import load_iris
iris = load_iris()
X = torch.tensor(iris["data"], dtype=torch.float32)
y = torch.tensor(iris["target"])
dataset = torch.utils.data.TensorDataset(X, y)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [100, 50])

In [35]:
a  = from_default_prior(ScaleMixturePrior())

In [36]:
a

BayesianConversionConfig(modules_to_replace={<class 'torch.nn.modules.linear.Linear'>: BayesianModuleConfig(module=<class 'src.bayesian.modules.BayesianLinear'>, priors={'weight': ScaleMixturePrior(), 'bias': ScaleMixturePrior()})})

In [12]:
inference = VariationalInference(net)

In [13]:
trainer = Trainer(max_epochs=500)
trainer.fit(
    inference,
    train_dataloaders=torch.utils.data.DataLoader(train_dataset, batch_size=25, shuffle=True),
    val_dataloaders=torch.utils.data.DataLoader(test_dataset, batch_size=25, shuffle=True),
)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name          | Type       | Params
---------------------------------------------
0 | model         | Net        | 166   
1 | train_metrics | ModuleDict | 0     
2 | val_metrics   | ModuleDict | 0     
---------------------------------------------
166       Trainable params
0         Non-trainable params
166       Total params
0.001     Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/soren/Repositories/Speciale/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:376: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for val/test/predict dataloaders.
  rank_zero_warn(
/Users/soren/Repositories/Speciale/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


AttributeError: 'VariationalInference' object has no attribute 'w_samples'

In [6]:
model.net[0].variational_parameters["weight"].rho

Parameter containing:
tensor([[-4.4588, -4.4540, -4.4615, -4.4584],
        [-4.4642, -4.4572, -4.4567, -4.4530],
        [-4.4612, -4.4575, -4.4579, -4.4553],
        [-4.4618, -4.4559, -4.4582, -4.4519],
        [-4.4645, -4.4584, -4.4611, -4.4510],
        [-4.4676, -4.4588, -4.4592, -4.4523],
        [-4.4623, -4.4556, -4.4582, -4.4544],
        [-4.4641, -4.4575, -4.4573, -4.4544],
        [-4.4621, -4.4561, -4.4596, -4.4553],
        [-4.4648, -4.4575, -4.4589, -4.4528]], requires_grad=True)

TypeError: 'module' object is not callable

In [283]:
x = torch.tensor([[1., 2., 3.], [20., 40, 60.]])
model = nn.Sequential

a = BayesianLinear.from_freq_module(nn.Linear(3, 5))
b = VariationalModule(a)

x_expanded = x.unsqueeze(0).expand((n_particles, *x.shape))

In [284]:
out = b(x_expanded, n_particles=n_particles)
loglik_standin = out.sum((1, 2))
elbo = (loglik_standin + b.log_prior() + b.log_v_post()).mean()
b.zero_grad()
elbo.backward()
b.variational_parameters["weight"].mu.grad

tensor([[23.4538, 38.5276, 65.9396],
        [22.1915, 40.8026, 65.3294],
        [17.1115, 41.4849, 66.6112],
        [23.3285, 40.7437, 63.9023],
        [24.6270, 43.1445, 66.6718]])

In [285]:
b.update_gradients_()
b.variational_parameters["weight"].mu.grad

tensor([[-32.1241, -20.8085,  55.0730],
        [-36.8658,  -0.7628,  58.0859],
        [-34.9798,  24.1958,  64.2660],
        [-17.7285,  -7.2025,  42.2222],
        [-23.5499,   5.1829,  60.1604]])

VariationalModule(
  (bayesian_module): BayesianLinear(
    (priors): ModuleDict(
      (weight): ScaleMixturePrior()
      (bias): ScaleMixturePrior()
    )
  )
  (variational_parameters): ModuleDict(
    (weight): Module()
    (bias): Module()
  )
)

In [50]:
from src.data.cifar import CIFARDataModule

In [51]:
dm = CIFARDataModule()
dm.setup()
train_loader = dm.train_dataloader()
X, y = next(iter(train_loader))

In [82]:
class WeHaveBatchNorm2dAtHome(torch.nn.Module):
    
    def __init__(self, scale, shift):
        super().__init__()
        self.register_parameter("scale", torch.nn.Parameter(scale.view(1, -1, 1, 1)))
        self.register_parameter("shift", torch.nn.Parameter(shift.view(1, -1, 1, 1)))
    def forward(self, x):
        return self.scale * x + self.shift

    @classmethod
    def from_batchnorm2d(cls, norm):
        a = 1/torch.sqrt(norm.running_var + norm.eps) * norm.weight.detach()
        b = -norm.running_mean*a + norm.bias.detach()
        return cls(a, b)



In [84]:
fakenorm = WeHaveBatchNorm2dAtHome.from_batchnorm2d(model.features.norm0)

In [103]:
fakenorm(model.features.conv0(X))

tensor([[[[ 7.9379e-02,  7.1383e-02,  6.8835e-02,  ...,  7.3523e-02,
            7.4663e-02,  8.0896e-02],
          [ 8.7760e-02,  8.4119e-02,  8.3099e-02,  ...,  8.5306e-02,
            8.4128e-02,  8.7573e-02],
          [ 9.0997e-02,  8.9074e-02,  8.9966e-02,  ...,  9.3295e-02,
            8.9264e-02,  8.9927e-02],
          ...,
          [ 1.0251e-01,  1.1618e-01,  1.2741e-01,  ...,  6.0238e-02,
            6.1609e-02,  7.0405e-02],
          [ 1.1096e-01,  1.2191e-01,  1.3123e-01,  ...,  7.8494e-02,
            8.6942e-02,  9.2082e-02],
          [ 1.0686e-01,  9.8975e-02,  8.4568e-02,  ...,  1.2511e-01,
            1.2446e-01,  1.1611e-01]],

         [[ 3.5888e-02,  3.7583e-02,  3.7414e-02,  ...,  3.9420e-02,
            3.9037e-02,  4.1402e-02],
          [ 3.9997e-02,  3.8029e-02,  3.8449e-02,  ...,  3.8528e-02,
            3.8738e-02,  3.8797e-02],
          [ 3.9698e-02,  3.8177e-02,  3.8644e-02,  ...,  4.1006e-02,
            3.8224e-02,  3.8121e-02],
          ...,
     

In [102]:
model.features[:2](X)

tensor([[[[ 7.2091e-02,  6.0702e-02,  5.7073e-02,  ...,  6.3750e-02,
            6.5373e-02,  7.4252e-02],
          [ 8.4029e-02,  7.8842e-02,  7.7389e-02,  ...,  8.0532e-02,
            7.8855e-02,  8.3762e-02],
          [ 8.8639e-02,  8.5900e-02,  8.7171e-02,  ...,  9.1912e-02,
            8.6170e-02,  8.7114e-02],
          ...,
          [ 1.0504e-01,  1.2451e-01,  1.4050e-01,  ...,  4.4827e-02,
            4.6780e-02,  5.9309e-02],
          [ 1.1707e-01,  1.3267e-01,  1.4594e-01,  ...,  7.0831e-02,
            8.2864e-02,  9.0184e-02],
          [ 1.1123e-01,  1.0000e-01,  7.9482e-02,  ...,  1.3722e-01,
            1.3630e-01,  1.2440e-01]],

         [[ 3.1965e-02,  3.4989e-02,  3.4689e-02,  ...,  3.8270e-02,
            3.7586e-02,  4.1807e-02],
          [ 3.9298e-02,  3.5787e-02,  3.6537e-02,  ...,  3.6678e-02,
            3.7052e-02,  3.7157e-02],
          [ 3.8765e-02,  3.6051e-02,  3.6885e-02,  ...,  4.1100e-02,
            3.6134e-02,  3.5950e-02],
          ...,
     

tensor([[[[ 7.9379e-02,  7.1383e-02,  6.8835e-02,  ...,  7.3523e-02,
            7.4663e-02,  8.0896e-02],
          [ 8.7760e-02,  8.4119e-02,  8.3099e-02,  ...,  8.5306e-02,
            8.4128e-02,  8.7573e-02],
          [ 9.0997e-02,  8.9074e-02,  8.9966e-02,  ...,  9.3295e-02,
            8.9264e-02,  8.9927e-02],
          ...,
          [ 1.0251e-01,  1.1618e-01,  1.2741e-01,  ...,  6.0238e-02,
            6.1609e-02,  7.0405e-02],
          [ 1.1096e-01,  1.2191e-01,  1.3123e-01,  ...,  7.8494e-02,
            8.6942e-02,  9.2082e-02],
          [ 1.0686e-01,  9.8975e-02,  8.4568e-02,  ...,  1.2511e-01,
            1.2446e-01,  1.1611e-01]],

         [[ 3.5888e-02,  3.7583e-02,  3.7414e-02,  ...,  3.9420e-02,
            3.9037e-02,  4.1402e-02],
          [ 3.9997e-02,  3.8029e-02,  3.8449e-02,  ...,  3.8528e-02,
            3.8738e-02,  3.8797e-02],
          [ 3.9698e-02,  3.8177e-02,  3.8644e-02,  ...,  4.1006e-02,
            3.8224e-02,  3.8121e-02],
          ...,
     

In [25]:
torch.nn.Sequential(
    model.features.conv0,
    
)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu